In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

In [2]:
batch_size,num_steps = 32,35
trainer,vocab = d2l.load_data_time_machine(batch_size,num_steps)

In [3]:
# 定义模型
num_hiddens = 256
rnn_layer = nn.RNN(len(vocab),num_hiddens)

In [4]:
# 使用张量来初始化隐藏状态
state = torch.zeros((1,batch_size,num_hiddens))
state.shape

torch.Size([1, 32, 256])

In [5]:
# 通过一个隐藏状态和一个输入，我们可以用更新后的隐藏状态计算输出
X = torch.rand(size=(num_steps,batch_size,len(vocab)))
Y,state_new = rnn_layer(X,state)
Y.shape,state_new.shape

(torch.Size([35, 32, 256]), torch.Size([1, 32, 256]))

In [6]:
class RNNModel(nn.Module):
    """循环神经网络模型。"""
    def __init__(self,rnn_layer,vocab_size,**kwargs):
        super(RNNModel,self).__init__(**kwargs):
        self.rnn = rnn_layer
        self.vocab_size = vocab_size
        self.num_hiddens = self.rnn.hidden_size
        if not self.rnn.bidirectional:
            self.num_directions = 1
            self.linear = nn.Linear(self.num_hiddens,self.vocab_size)
        else:
            self.num_directions = 2
            self.linear = nn.Linear(self.num_hiddens*2,self.vocab_size)
    def forward(self,inputs,state):
        X = F.one_hot(inputs.T.long(),self.vocab_size)
        X = X.to(torch.float32)
        Y,state = self.rnn(X,state)
        output = self.linear(Y.reshape((-1,Y.shape[-1])))
        return output,state
    def begin_state(self,device,batch_size =1):
        if not isinstance(self.rnn,nn.LSTM):
            return torch.zeros((self.num_directions*self.rnn))

SyntaxError: invalid syntax (1995595138.py, line 4)